In [397]:
import psycopg2
import pandas as pd
from utils import *

In [398]:
conn = psycopg2.connect(
    host = "codd04.research.northwestern.edu",
    database = "postgres",
    user = "cpdbstudent",
    password = "DataSci4AI",
    port = 5433
)
cursor = conn.cursor()

In [399]:
cursor.execute("""
    SELECT * FROM trr_trr_refresh;
""")
results = cursor.fetchall()
colnames = [desc[0] for desc in cursor.description]

In [400]:
df = pd.DataFrame(results, columns=colnames)
df

,id,event_number,cr_number,beat,block,direction,street,location,trr_datetime,indoor_or_outdoor,...,officer_race,officer_age,officer_appointed_date,officer_birth_year,officer_unit_name,officer_unit_detail,trr_created,latitude,longitude,point
0,96630,1611411724,None,1124,30XX,West,JACKSON BLVD,STREET,2016-04-23 18:18:00-05,None,...,WHITE,30,02-18-14,None,11,None,None,41.8776497,-87.7012653,0101000020E61000007554DA87E1EC55C00F6C4BD356F0...
1,97999,1618611539,None,723,9XX,West,MARQUETTE RD,STREET,2016-07-04 17:06:00-05,Outdoor,...,WHITE,30,11-25-13,None,7,None,None,41.7724444,-87.6470302,0101000020E61000008F215BF168E955C03AFD4575DFE2...
2,98796,1623610044,None,811,53XX,South,NARRAGANSETT AVE,RESIDENCE,2016-08-23 16:14:00-05,Outdoor,...,WHITE,51,09-30-02,None,8,None,None,41.7959057,-87.7821939,0101000020E61000005DE800770FF255C09919EC3CE0E5...
3,99293,1626900073,None,2232,100XX,South,EMERALD AVE,SIDEWALK,2016-09-25 20:00:00-05,None,...,WHITE,34,12-16-09,None,22,None,None,41.7122367,-87.6421715,0101000020E610000017BB7D5619E955C067C17A922ADB...
4,99369,1627409687,None,634,XX,West,95TH ST,SIDEWALK,2016-09-30 17:13:00-05,None,...,BLACK,46,02-21-06,None,6,None,None,41.72199,-87.626064,0101000020E6100000FD4CBD6E11E855C0FF04172B6ADC...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17460,6043642,262,None,2534,12XX,N,KILDARE AVE,RESIDENCE - PORCH / HALLWAY,2020-05-26 00:30:00-05,INDOOR,...,HISPANIC,-25,2019-FEB-19,1995,25,None,2020-05-26 03:42:00-05,41.9027893,-87.733746,0101000020E61000008864C8B1F5EE55C0E1568B998EF3...
17461,6043651,262,None,2534,12XX,N,KILDARE AVE,RESIDENCE - PORCH / HALLWAY,2020-05-26 00:30:00-05,INDOOR,...,HISPANIC,-24,2019-AUG-16,1996,25,None,2020-05-26 03:45:00-05,41.9027893,-87.733746,0101000020E61000008864C8B1F5EE55C0E1568B998EF3...
17462,6059341,2014902656,None,1023,26XX,W,21ST ST,APARTMENT,2020-05-28 05:20:00-05,INDOOR,...,HISPANIC,-31,2017-SEP-18,1989,10,None,2020-05-28 06:52:00-05,41.8540554,-87.6908109,0101000020E61000001ACEEB3E36EC55C071FCF5AF51ED...
17463,6059467,2014902656,None,1023,26XX,W,21ST ST,APARTMENT,2020-05-28 05:20:00-05,INDOOR,...,HISPANIC,-26,2018-NOV-27,1994,10,None,2020-05-28 07:24:00-05,41.8540554,-87.6908109,0101000020E61000001ACEEB3E36EC55C071FCF5AF51ED...


In [401]:
# convert to integers
df['beat'] = to_int(df['beat'])
df['officer_birth_year'] = to_int(df['officer_birth_year'])
df['officer_age'] = to_int(df['officer_age'])
df['subject_birth_year'] = to_int(df['subject_birth_year'])
df['subject_age'] = to_int(df['subject_age'])

In [402]:
# convert to booleans
to_bool(df, 'officer_on_duty')
to_bool(df, 'officer_injured')
to_bool(df, 'officer_in_uniform')
to_bool(df, 'subject_armed')
to_bool(df, 'subject_injured')
to_bool(df, 'subject_alleged_injury')
to_bool(df, 'notify_oemc')
to_bool(df, 'notify_district_sergeant')
to_bool(df, 'notify_op_command')
to_bool(df, 'notify_det_division')

In [403]:
# reconcile races of subject and officer
reconcile_race(df, 'subject_race')
reconcile_race(df, 'officer_race')

In [404]:
# convert to datetime
df['trr_datetime'] = pd.to_datetime(df['trr_datetime'])
df['trr_created'] = pd.to_datetime(df['trr_created'])

In [405]:
# remove redacted
df['officer_appointed_date'].replace('REDACTED', None, inplace=True)
# convert to dates
df['officer_appointed_date'] = pd.to_datetime(df['officer_appointed_date'])

In [406]:
# capitalize first name
df['officer_first_name'] = df['officer_first_name'].str.title()
# capitalize last name
df['officer_last_name'] = df['officer_last_name'].str.title()
# TODO remove suffix from last name

In [407]:
df['subject_gender'].replace('MALE', 'M', inplace=True)
df['subject_gender'].replace('FEMALE', 'F', inplace=True)

In [408]:
df['subject_birth_year'] = to_int(df['subject_birth_year'])
df['subject_birth_year'] = df['subject_birth_year'].apply(reconcile_subject_birth_year)

In [409]:
pd.unique(df['subject_birth_year'])

array([1989, 1997, 1983, 1985, 1994, 1982, 1995, 1960, 1996, 1991, 1993,
       1935, 1987, 1980, 1971, 1972, 1979, 1978, 1975, 1986, 1071, 1968,
       1984, 1988, 1981, 1998, 1990, 1963, 1957, 1958, 1956, 1962, 1966,
       1965, 1970, 1955, 1977, 1992, 1947, 1961, 1969, 1964, 1976, 1937,
       1954, 1974, 1065, 1973, 1092, 1929, 1087, 1019, 1959, 1952, 1948,
       1967, 1078, 1951, 1949, 1095, 1900, 1950, 1901, 1094, 1999, 1953,
       1091, 2001, 1093, 1066, 1944, 1067, 1073, 2000, 1902, 1942, 1930,
       1922, 2002, 1940,  198, 1082, 1194, 1088, 1097, 1918, 1080])

In [410]:
# cleanup indoor or outdoor
df['indoor_or_outdoor'].replace('OUTDOOR', 'Outdoor', inplace=True)
df['indoor_or_outdoor'].replace('INDOOR', 'Indoor', inplace=True)

In [411]:
# convert all locations to capital case
df['location'] = df['location'].str.title()

In [412]:
# formatting of locations
df['location'].replace('Cha Parking Lot / Grounds', 'Cha Parking Lot/Grounds', inplace=True)
df['location'].replace('Cha Hallway / Stairwell / Elevator', 'Cha Hallway/Stairwell/Elevator', inplace=True)
df['location'].replace('Church / Synagogue / Place Of Worship', 'Church/Synagogue/Place Of Worship', inplace=True)
df['location'].replace('College / University - Grounds', 'College/University Grounds', inplace=True)
df['location'].replace('Cta Parking Lot / Garage / Other Property', 'Cta Garage / Other Property', inplace=True)
df['location'].replace('School - Private Building', 'School, Private, Building', inplace=True)
df['location'].replace('School - Private Grounds', 'School, Private, Grounds', inplace=True)
df['location'].replace('School - Public Building', 'School, Public, Building', inplace=True)
df['location'].replace('School - Public Grounds', 'School, Public, Grounds', inplace=True)
df['location'].replace('Medical / Dental Office', 'Medical/Dental Office', inplace=True)
df['location'].replace('Hospital Building / Grounds', 'Hospital Building/Grounds', inplace=True)
df['location'].replace('Hotel / Motel', 'Hotel/Motel', inplace=True)
df['location'].replace('Highway / Expressway', 'Highway/Expressway', inplace=True)
df['location'].replace('Lakefront / Waterfront / Riverbank', 'Lakefront/Waterfront/Riverbank', inplace=True)
df['location'].replace('Movie House / Theater', 'Movie House/Theater', inplace=True)
df['location'].replace('Nursing / Retirement Home', 'Nursing Home/Retirement Home', inplace=True)
df['location'].replace('Vehicle - Commercial', 'Vehicle-Commercial', inplace=True)
df['location'].replace('Vehicle - Other Ride Share Service (Lyft, Uber, Etc.)', 'Vehicle - Other Ride Service', inplace=True)
df['location'].replace('Vacant Lot / Land', 'Vacant Lot/Land', inplace=True)
df['location'].replace('Tavern / Liquor Store', 'Tavern/Liquor Store', inplace=True)
df['location'].replace('Parking Lot / Garage (Non Residential)', 'Parking Lot/Garage(Non.Resid.)', inplace=True)
df['location'].replace('Police Facility / Vehicle Parking Lot', 'Police Facility/Veh Parking Lot', inplace=True)
df['location'].replace('Sports Arena / Stadium', 'Sports Arena/Stadium', inplace=True)
df['location'].replace('Residence - Garage', 'Residence-Garage', inplace=True)
df['location'].replace('Residence - Porch / Hallway', 'Residence Porch/Hallway', inplace=True)
df['location'].replace('Residence - Yard (Front / Back)', 'Residential Yard (Front/Back)', inplace=True)
df['location'].replace('Factory / Manufacturing Building', 'Factory/Manufacturing Building', inplace=True)
df['location'].replace('Government Building / Property', 'Government Building/Property', inplace=True)
df['location'].replace('Other (Specify)', 'Other', inplace=True)
df['location'].replace('Other Railroad Property / Train Depot', 'Other Railroad Prop / Train Depot', inplace=True)
df['location'].replace('Airport Parking Lot', 'Parking Lot/Garage(Non.Resid.)', inplace=True)

In [413]:
df['street'] = df['street'].str.title()